In [1]:
import cmath
import numpy as np
from math import log, ceil
import pylab as plt
import torch
import cv2
import os
from sklearn.preprocessing import normalize
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def calculate_fourier_spectrum(im):
    im = im.float()
    im = im.cpu()
    im = im.data.numpy() #transorm to numpy
    fft = np.fft.fft2(im)
    fourier_spectrum = np.abs(fft)
    return fourier_spectrum

def calculate_spectra(images):
    fs = []   
    for i in range(len(images)):
        image = images[i]
        fourier_image = calculate_fourier_spectrum(image)
        fs.append(fourier_image.flatten())
    return fs
    

In [6]:
im_size = (350, 225)

In [7]:
p = []
dir = "drive/MyDrive/data (1)/"
files = os.listdir(dir)
for i in files:
  x = cv2.imread(dir+i)
  x = cv2.resize(x, im_size)
  p.append(torch.from_numpy(x).reshape((3, im_size[0], im_size[1])))
# p = np.array(p)
# norm1 = (p-np.min(p))/(np.max(p)-np.min(p))
#print(norm1.min())
#type(norm1)


In [8]:
ap = []
dir = "/content/drive/MyDrive/ML_Sec_AD/"
files= os.listdir(dir)
for i in files:
  x = cv2.imread(dir+i)
  x1 = cv2.resize(x, im_size)
  del x
  ap.append(torch.from_numpy(x1).reshape((3, im_size[0], im_size[1])))
  
#ap = np.array(ap)
#print(ap)
#norm2 = (ap-np.min(ap))/(np.max(ap)-np.min(ap))
#norm2.max()

In [9]:
print(ap[0].shape)
len(ap)

torch.Size([3, 350, 225])


10

In [10]:
mfs = calculate_spectra(p)
mfs_advs = calculate_spectra(ap)
characteristics       = np.asarray(mfs, dtype=np.float32)
characteristics_adv   = np.asarray(mfs_advs, dtype=np.float32)

In [11]:
characteristics.shape

(660, 236250)

In [12]:
shape1 = np.shape(characteristics)
k1 = shape1[0]
shape2 = np.shape(characteristics_adv)
k2 = shape2[0]

In [13]:
zer= [0]*(k1)
one = [1] *k2
len(zer)
zer = np.array(zer)
one = np.array(one)

In [14]:
b_X_train_val, b_X_test, b_y_train_val, b_y_test = train_test_split(characteristics, zer, test_size=0.2, random_state=42)

In [15]:
adv_X_train_val, adv_X_test, adv_y_train_val, adv_y_test = train_test_split(characteristics_adv, one, test_size=0.2, random_state=42)

In [16]:
print(b_X_train_val.shape)
print(b_X_test.shape)
print(b_y_train_val.shape)
print(b_y_test.shape)

(528, 236250)
(132, 236250)
(528,)
(132,)


In [17]:
print(adv_X_train_val.shape)
print(adv_X_test.shape)
print(adv_y_train_val.shape)
print(adv_y_test.shape)

(8, 236250)
(2, 236250)
(8,)
(2,)


In [18]:
b_X_train, b_X_val, b_y_train, b_y_val = train_test_split(b_X_train_val, b_y_train_val, test_size=0.2, random_state=42)

In [19]:
adv_X_train, adv_X_val, adv_y_train, adv_y_val = train_test_split(adv_X_train_val, adv_y_train_val, test_size=0.2, random_state=42)


In [20]:
X_train = np.concatenate(( b_X_train,adv_X_train))
y_train = np.concatenate(( b_y_train,adv_y_train))

In [21]:
X_test = np.concatenate(( b_X_test, adv_X_test))
y_test = np.concatenate(( b_y_test,adv_y_test))

In [23]:
rf = RandomForestClassifier(n_estimators = 50, max_features = "auto", random_state = 42)
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50, random_state=42)

In [24]:
predictions = rf.predict(X_test)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1])

In [26]:
accuracy = (y_test == predictions).sum() / y_test.shape[0]

In [27]:
accuracy

0.9925373134328358